## Imports

In [139]:
import torch
from pathlib import Path
import matplotlib.pyplot as plt

## Load dataset

In [124]:
names = Path("../names.txt").read_text().split("\n")
print(f"There are {len(names)} names in the dataset")
print(f"First 10 are:\n{names[:10]}")

There are 32033 names in the dataset

First 10 are:
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']

## Prepare the character-level tokenizer

In [125]:
tokenizer = {s:i for (i,s) in enumerate(".abcdefghijklmnopqrstuvwxyz")}
print(tokenizer)
detokenizer = {i:s for (s,i) in tokenizer.items()}
print(detokenizer)

{
    '.': 0,
    'a': 1,
    'b': 2,
    'c': 3,
    'd': 4,
    'e': 5,
    'f': 6,
    'g': 7,
    'h': 8,
    'i': 9,
    'j': 10,
    'k': 11,
    'l': 12,
    'm': 13,
    'n': 14,
    'o': 15,
    'p': 16,
    'q': 17,
    'r': 18,
    's': 19,
    't': 20,
    'u': 21,
    'v': 22,
    'w': 23,
    'x': 24,
    'y': 25,
    'z': 26
}

{
    0: '.',
    1: 'a',
    2: 'b',
    3: 'c',
    4: 'd',
    5: 'e',
    6: 'f',
    7: 'g',
    8: 'h',
    9: 'i',
    10: 'j',
    11: 'k',
    12: 'l',
    13: 'm',
    14: 'n',
    15: 'o',
    16: 'p',
    17: 'q',
    18: 'r',
    19: 's',
    20: 't',
    21: 'u',
    22: 'v',
    23: 'w',
    24: 'x',
    25: 'y',
    26: 'z'
}

## Prepare the training dataset

In [227]:
training_input = []
training_output = []
for name in names:
    name = "." + name + "."
    for ch1, ch2 in zip(name, name[1:]):
        training_input.append(tokenizer[ch1])
        training_output.append(tokenizer[ch2])

training_input = torch.nn.functional.one_hot(torch.tensor(training_input), num_classes=27).float()
training_output = torch.tensor(training_output)
training_output_onehot = torch.nn.functional.one_hot(training_output, num_classes=27).float()

In [195]:
print(training_input.shape, training_input.dtype)

torch.Size([228146, 27])
torch.float32

In [228]:
print(training_output.shape)
print(training_output_onehot.shape)

torch.Size([228146])

torch.Size([228146, 27])

## Initialize random weights

In [233]:
W = torch.randn((27,27), dtype=torch.float32, requires_grad=True)

## Training

In [234]:
for _ in range(100):
    actual = training_input @ W
    loss = torch.nn.functional.cross_entropy(actual, training_output_onehot)
    loss.backward()
    W.data += -0.1 * W.grad
print(loss)
for _ in range(100):
    actual = training_input @ W
    loss = torch.nn.functional.cross_entropy(actual, training_output_onehot)
    loss.backward()
    W.data += -0.01 * W.grad
print(loss)

tensor(2.7816, grad_fn=<DivBackward1>)

tensor(2.6854, grad_fn=<DivBackward1>)

## Sampling

In [239]:
for _ in range(10):
    generated = None
    name = "."
    while generated != '.':
        actual = torch.nn.functional.one_hot(torch.tensor(tokenizer[generated or "."]), num_classes=27).float() @ W
        actual = actual.exp()
        P = actual / actual.sum()
        idx = torch.multinomial(P, 1)
        generated = detokenizer[idx.item()]
        name += generated
    print(name)

.kenn.

.e.

.g.

.ruole.

.zwceles.

.jala.

.sha.

.an.

.zx.

.ale.